In [ ]:
import intake
import pandas as pd

import holoviews as hv
from holoviews import opts
from holoviews.streams import Params
import hvplot.xarray
import hvplot.pandas

import colorcet as cc
import panel as pn

import geoviews as gv
from geoviews.tile_sources import EsriImagery, tile_sources

hv.extension('bokeh')

In [ ]:
mean_temp_f = intake.open_zarr('./data/mean_temp.zarr').read()
trees_df = pd.read_parquet('./data/trees.parq')

In [ ]:
special_cmap = cc.fire[::-1][90:]

In [ ]:
p = (mean_temp_f.hvplot(x='x', y='y', geo=True, height=500, width=550, 
                        cmap=special_cmap, alpha=.5, legend=False)
                .relabel('Mean Surface Temp (F)'))
options = opts.Image(clipping_colors={'min': 'transparent', 'max': 'transparent'})
clipped = p.opts(options, clone=True)

trees_p = trees_df.hvplot.points('Longitude', 'Latitude', title='Street Tree Density',
                                 geo=True, datashade=True, dynspread=True,
                                 height=500, width=500, legend=False, cmap=cc.kgy[::-1])

start, end = (int(mean_temp_f.temperature.min() - 1), 
              int(mean_temp_f.temperature.max() + 1))
thresh_slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                       step=1, bar_color='#ff0000',
                                       name='Temperature range')
thresh_slider.jslink(clipped, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

temp_alpha_slider = pn.widgets.FloatSlider(start=0, end=1, value=.5, name='Temperature alpha')
temp_alpha_slider.jslink(clipped, value='glyph.global_alpha')


trees_toggle = pn.widgets.Toggle(name='Tree Layer', value=True)
trees_toggle.jslink(trees_p, value='glyph_renderer.visible')

trees_alpha_slider = pn.widgets.FloatSlider(start=0, end=1, value=.7, name='Trees alpha')
trees_alpha_slider.jslink(trees_p, value='glyph.global_alpha')

basemap_selector = pn.widgets.Select(name='Tiling Service', value=EsriImagery, options=tile_sources)

def set_tile_source(value):
    return value
    
dmap = hv.DynamicMap(set_tile_source, streams=[Params(basemap_selector, ['value'])])

dashboard = pn.Row(
    pn.Column(
        '# Heat and Trees',
        thresh_slider, 
        temp_alpha_slider,
        basemap_selector,
        trees_toggle,
        trees_alpha_slider),
    clipped.opts(framewise=False) * trees_p.opts(framewise=False) * dmap.opts(framewise=False)
)

In [ ]:
dashboard.servable()